# Get an example dataset

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import os

import tensorflow as tf
from tensorflow import keras

tf.__version__

W0809 00:18:12.228423  3068 __init__.py:687] 

  TensorFlow's `tf-nightly` package will soon be updated to TensorFlow 2.0.

  Please upgrade your code to TensorFlow 2.0:
    * https://www.tensorflow.org/beta/guide/migration_guide

  Or install the latest stable TensorFlow 1.X release:
    * `pip install -U "tensorflow==1.*"`

  Otherwise your code may be broken by the change.

  


'1.15.0-dev20190803'

In [2]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

11493376/11490434 [==============================] - 2s 0us/step


# Define a model

In [3]:
# Returns a short sequential model
def create_model():
  model = tf.keras.models.Sequential([
    keras.layers.Dense(512, activation=tf.keras.activations.relu, input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10, activation=tf.keras.activations.softmax)
  ])

  model.compile(optimizer=tf.keras.optimizers.Adam(),
                loss=tf.keras.losses.sparse_categorical_crossentropy,
                metrics=['accuracy'])

  return model


# Create a basic model instance
model = create_model()
model.summary()

W0809 00:27:35.311535  3068 deprecation.py:506] From C:\Users\nguyen.pv162992\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow_core\python\ops\resource_variable_ops.py:1633: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [4]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create checkpoint callback
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

model = create_model()

model.fit(train_images, train_labels,  epochs = 10,
          validation_data = (test_images,test_labels),
          callbacks = [cp_callback])  # pass callback to training

# This may generate warnings related to saving the state of the optimizer.
# These warnings (and similar warnings throughout this notebook)
# are in place to discourage outdated usage, and can be ignored.

W0809 00:34:14.916467  3068 deprecation.py:323] From C:\Users\nguyen.pv162992\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow_core\python\keras\optimizer_v2\optimizer_v2.py:468: BaseResourceVariable.constraint (from tensorflow.python.ops.resource_variable_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Apply a constraint manually following the optimizer update step.


Train on 1000 samples, validate on 1000 samples
Epoch 1/10
 992/1000 [============================>.] - ETA: 0s - loss: 1.1919 - acc: 0.6512
Epoch 00001: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 1s 522us/sample - loss: 1.1860 - acc: 0.6530 - val_loss: 0.6959 - val_acc: 0.7910
Epoch 2/10
 928/1000 [==========================>...] - ETA: 0s - loss: 0.4198 - acc: 0.8890
Epoch 00002: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 0s 350us/sample - loss: 0.4181 - acc: 0.8880 - val_loss: 0.5196 - val_acc: 0.8460
Epoch 3/10
 800/1000 [=======================>......] - ETA: 0s - loss: 0.2407 - acc: 0.9362
Epoch 00003: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 0s 414us/sample - loss: 0.2803 - acc: 0.9280 - val_loss: 0.4946 - val_acc: 0.8590
Epoch 4/10
 928/1000 [==========================>...] - ETA: 0s - loss: 0.2255 - acc: 0.9332
Epoch 00004: saving model to training_1/cp.ckpt
1000/10

In [6]:
model = create_model()

loss, acc = model.evaluate(test_images, test_labels)
print("Untrained model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 104us/sample - loss: 2.3750 - acc: 0.0920
Untrained model, accuracy:  9.20%


In [7]:
model.load_weights(checkpoint_path)
loss,acc = model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 72us/sample - loss: 0.4486 - acc: 0.8640
Restored model, accuracy: 86.40%


In [8]:
# include the epoch in the file name. (uses `str.format`)
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_path, verbose=1, save_weights_only=True,
    # Save weights, every 5-epochs.
    period=5)

model = create_model()
model.save_weights(checkpoint_path.format(epoch=0))
model.fit(train_images, train_labels,
          epochs = 50, callbacks = [cp_callback],
          validation_data = (test_images,test_labels),
          verbose=0)

W0809 00:35:51.974286  3068 callbacks.py:862] `period` argument is deprecated. Please use `save_freq` to specify the frequency in number of samples seen.



Epoch 00005: saving model to training_2/cp-0005.ckpt

Epoch 00010: saving model to training_2/cp-0010.ckpt

Epoch 00015: saving model to training_2/cp-0015.ckpt

Epoch 00020: saving model to training_2/cp-0020.ckpt

Epoch 00025: saving model to training_2/cp-0025.ckpt

Epoch 00030: saving model to training_2/cp-0030.ckpt

Epoch 00035: saving model to training_2/cp-0035.ckpt

Epoch 00040: saving model to training_2/cp-0040.ckpt

Epoch 00045: saving model to training_2/cp-0045.ckpt

Epoch 00050: saving model to training_2/cp-0050.ckpt


In [10]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2\\cp-0050.ckpt'

In [11]:
model = create_model()
model.load_weights(latest)
loss, acc = model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 109us/sample - loss: 0.5013 - acc: 0.8680
Restored model, accuracy: 86.80%


In [12]:
# Save the weights
model.save_weights('./checkpoints/my_checkpoint')

# Restore the weights
model = create_model()
model.load_weights('./checkpoints/my_checkpoint')

loss,acc = model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 130us/sample - loss: 0.5013 - acc: 0.8680
Restored model, accuracy: 86.80%


In [13]:
model = create_model()

model.fit(train_images, train_labels, epochs=5)

# Save entire model to a HDF5 file
model.save('my_model.h5')

Train on 1000 samples
Epoch 1/5
1000/1000 [==============================] - 0s 269us/sample - loss: 1.1090 - acc: 0.6920
Epoch 2/5
1000/1000 [==============================] - 0s 197us/sample - loss: 0.4363 - acc: 0.8750
Epoch 3/5
1000/1000 [==============================] - 0s 197us/sample - loss: 0.2865 - acc: 0.9300
Epoch 4/5
1000/1000 [==============================] - 0s 201us/sample - loss: 0.2050 - acc: 0.9470
Epoch 5/5
1000/1000 [==============================] - 0s 196us/sample - loss: 0.1475 - acc: 0.9650


In [14]:
# Recreate the exact same model, including weights and optimizer.
new_model = keras.models.load_model('my_model.h5')
new_model.summary()

W0809 00:41:22.987051  3068 deprecation.py:506] From C:\Users\nguyen.pv162992\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow_core\python\ops\init_ops.py:97: calling GlorotUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0809 00:41:22.989047  3068 deprecation.py:506] From C:\Users\nguyen.pv162992\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow_core\python\ops\init_ops.py:97: calling Zeros.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_6 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [15]:
loss, acc = new_model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 138us/sample - loss: 0.4107 - acc: 0.8770
Restored model, accuracy: 87.70%


In [16]:
model = create_model()

model.fit(train_images, train_labels, epochs=5)

W0809 00:41:39.948801  3068 util.py:144] Unresolved object in checkpoint: (root).optimizer.iter
W0809 00:41:39.948801  3068 util.py:144] Unresolved object in checkpoint: (root).optimizer.beta_1
W0809 00:41:39.949803  3068 util.py:144] Unresolved object in checkpoint: (root).optimizer.beta_2
W0809 00:41:39.950801  3068 util.py:144] Unresolved object in checkpoint: (root).optimizer.decay
W0809 00:41:39.952798  3068 util.py:144] Unresolved object in checkpoint: (root).optimizer.learning_rate
W0809 00:41:39.953794  3068 util.py:144] Unresolved object in checkpoint: (root).optimizer's state 'm' for (root).layer_with_weights-0.kernel
W0809 00:41:39.954791  3068 util.py:144] Unresolved object in checkpoint: (root).optimizer's state 'm' for (root).layer_with_weights-0.bias
W0809 00:41:39.955789  3068 util.py:144] Unresolved object in checkpoint: (root).optimizer's state 'm' for (root).layer_with_weights-1.kernel
W0809 00:41:39.957783  3068 util.py:144] Unresolved object in checkpoint: (root).o

Train on 1000 samples
Epoch 1/5
1000/1000 [==============================] - 0s 285us/sample - loss: 1.1592 - acc: 0.6790
Epoch 2/5
1000/1000 [==============================] - 0s 272us/sample - loss: 0.4127 - acc: 0.8890
Epoch 3/5
1000/1000 [==============================] - 0s 203us/sample - loss: 0.2814 - acc: 0.9270
Epoch 4/5
1000/1000 [==============================] - 0s 234us/sample - loss: 0.2024 - acc: 0.9540
Epoch 5/5
1000/1000 [==============================] - 0s 225us/sample - loss: 0.1542 - acc: 0.9600


In [17]:
import time

saved_model_path = "./saved_models/"+str(int(time.time()))
tf.contrib.saved_model.save_keras_model(model, saved_model_path)

W0809 00:41:51.533660  3068 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

W0809 00:41:55.329408  3068 deprecation.py:323] From <ipython-input-17-998dfe524d77>:4: export_saved_model (from tensorflow.python.keras.saving.saved_model_experimental) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `model.save(..., save_format="tf")` or `tf.keras.models.save_model(..., save_format="tf")`.
W0809 00:41:56.492627  3068 deprecation.py:323] From C:\Users\nguyen.pv162992\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow_core\python\saved_model\signature_def_utils_impl.py:253: build_tensor_info (from tensorflow.

In [18]:
!ls {saved_model_path}

'ls' is not recognized as an internal or external command,
operable program or batch file.


In [19]:
new_model = tf.contrib.saved_model.load_keras_model(saved_model_path)
new_model.summary()

W0809 00:42:06.837516  3068 deprecation.py:323] From <ipython-input-19-f5ba023fc0b4>:1: load_from_saved_model (from tensorflow.python.keras.saving.saved_model_experimental) is deprecated and will be removed in a future version.
Instructions for updating:
The experimental save and load functions have been  deprecated. Please switch to `tf.keras.models.load_model`.


Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_14 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_7 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_15 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [20]:
# The model has to be compiled before evaluating.
# This step is not required if the saved model is only being deployed.

new_model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.sparse_categorical_crossentropy,
              metrics=['accuracy'])

# Evaluate the restored model.
loss, acc = new_model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 147us/sample - loss: 0.4396 - acc: 0.8580
Restored model, accuracy: 85.80%
